<a href="https://colab.research.google.com/github/joyfulspace/ADP/blob/master/07_%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5_7_%EC%9C%A0%EC%A0%84%EC%9E%90%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 유전자알고리즘
* 모든 규칙을 만족시켜야 할 때 사용
* 교차(crossover): 2개의 염색체를 유합하는 방법 (염색체: 솔루션을 추상화한 정보 단위)
  - 교차점 1개: 싱글 크로스오버
  - 교차점이 많다고 꼭 좋은 해를 얻는 것은 아니고 다양한 해를 얻을 수 있음
* 변이(mutation): 돌연변이 발생할 때 오히려 좋은 해가 만들어질 수 있음.
  - 지역 최적화 탈출하는 효과 있음
  - 하나의 염색체, 정보 단위에서 랜덤으로 하나의 연결점을 찾은 후, 그 부분의 정보를 의도적으로 왜곡시키는 것

* 프로세스
  1. 문제/해/해집합 정의
    - 도메인 결정도 중요
    - 도메인: 해 집합의 범위
  2. 해(솔루션)의 염색체 정의 
  3. 적합도 함수 f(x)설계 
    - 적합도 평가
    - cost function 사용: 규칙 위배에 따른 상벌 부과
    - 환경에 적응하지 못하는 염색체는 페널티가 부과되어 자연선택의 확률이 낮아져 도태되도록 되어 있음
    - 규칙 정리하고 > 패널티, 리워드 정하고 > cost function 설계
    - 이 부분에 따라 성능이 크게 달라짐
    - 우선순위 정리가 필요
    - 모든 것을 100% 만족시키는 방법은 없음
  4. 좋은 품종의 유전자 교배(교차/변이 등) 
  5. 적합한 염색체 선별 
  6. 충분한 세대교체



In [10]:
# -*- coding: utf-8 -*-

import time
import random
import math
import scipy.stats
import numpy as np

# 구간과 도메인의 정의
times = ['A', 'B', 'C', 'D', 'E', 'F']
domain = [(0, (len(times)*3)-i-1) for i in range(0, len(times)*3)]

# cost function 설계를 위한 제약조건 정리
prefs = {
    1: ['B', 'C', 'A'], # 1번 사람은 B, C, A 시간대를 선호
    7: ['E', 'F'],
    11: ['A', 'B'],
    15: ['E', 'D']
}

hates = {13: [1, 7], # 13번 사람은 1번, 7번 사람과 함께 있기 싫음
         10: [11, 15],
         3: [8, 12, 10]
}

man = [range(0, 12)] # 0~11이 남자
woman = [range(12, 18)]

In [5]:
domain

[(0, 17),
 (0, 16),
 (0, 15),
 (0, 14),
 (0, 13),
 (0, 12),
 (0, 11),
 (0, 10),
 (0, 9),
 (0, 8),
 (0, 7),
 (0, 6),
 (0, 5),
 (0, 4),
 (0, 3),
 (0, 2),
 (0, 1),
 (0, 0)]

In [14]:
def dormcost(vec):
  # cost function 정의
  # 선호하는 시간대에 따라 점수 부여
  # 싫어하는 사람에 따라 점수 부여
  # 각 시간대에 적어도 1명은 남자
  # C시간대에 여성이 있으면 안 됨

  cost = 0

  # Create list a lot of slots
  slots = []
  for i in range(len(times)): slots+=[i,i,i]

  # 해 vec에 따른 시간대 설정
  times_band = printsolution(vec) #vec는 DNA값

  # 선호하는 시간대에 따라 점수 부여
  for i in prefs.keys():
    time = times_band[i]
    pref = prefs[i]

    satisfy = False
    for j in range(len(pref)):
      if pref[j]==time:
        cost += (j*2)
        satisfy = True
    if satisfy==False:
      cost += (j*2+3)

  # 싫어하는 사람에 따라 점수 부여
  for i in hates.keys():
    time = times_band[i]
    ha = hates[i]

    for j in range(len(ha)):
      y = int(vec[ha[j]])
      time_y = times[slots[y]]
      if time_y==time:
        cost += (7-j)

  # 각 방에 적어도 1명은 남자
  idx = 0
  time_dic = {'A':0, 'B':0, 'C':0, 'D':0, 'E':0, 'F':0}

  for t in times_band:
    idx += 1
    if idx>=0 and idx<=11:
      if t=='A': time_dic['A'] += 1
      elif t=='B': time_dic['B'] += 1
      elif t=='C': time_dic['C'] += 1
      elif t=='D': time_dic['D'] += 1
      elif t=='E': time_dic['E'] += 1
      elif t=='F': time_dic['F'] += 1

  # 페널티를 2점씩 부과해준다.
  cost += sum(np.matrix(time_dic.values())==0)*2

  # C 시간대에 여성이 있으면 안됨
  idx = 0
  time_dic = {'A':0, 'B':0, 'C':0, 'D':0, 'E':0, 'F':0}

  for t in times_band:
    idx += 1
    if idx>=12:
      if t=='C': time_dic['C'] += 1
  
  if time_dic['C']>0:
    cost += 10 # 패널티 10점 부과. 패널티 크기에 따라 우선순위 조정 가능

  return cost

In [6]:
# TEST
slots = []
for i in range(len(times)): slots+=[i,i,i]
slots

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5]

In [16]:
def printsolution(vec):
  slots = []
  times_band = []

  # 슬롯초기화
  for i in range(len(times)): slots+=[i,i,i]

  # 루프를 돌면서 전직원들의 시가배정
  for i in range(len(vec)):
    x = int(vec[i])

    # 직원의 시간대를 정한다.
    time = times[slots[x]]

    # 시간대를 추가함.
    times_band.append(time)

    # 해당하는 슬롯삭제(이미 배정되었으므로)
    del slots[x]

  return times_band

# 유전자 알고리즘의 핵심. 거의 변화 없이 다른 문제에도 적용 가능
def geneticoptimize(domain, costf=dormcost, popsize=200, step=1, mutprob=0.5, elite=0.4, maxiter=20):
  # popsize: 해집합 크기, elite: 좋은 해 비율 0.4로 해서 좋은 해끼리 교배. mutprob: 변이 확률, step: 변이할 때 변화되는 정도
  # maxiter: 세대수
  
  # 변이
  def mutate(vec):
    i = random.randint(0, len(domain)-1)
    if random.random()<0.5 and vec[i]-step>domain[i][0]:
      return vec[0:i]+[vec[i]-step]+vec[i+1:]
    elif vec[i]+step<domain[i][1]:
      return vec[0:i]+[vec[i]+step]+vec[i+1:]
    else:
      return vec

  #교차연산
  def crossover(r1, r2):
    i = random.randint(1, len(domain)-1)
    return r1[0:i] + r2[i:] # 새로운 해 만듬

  # 초기의 random 해 설정
  pop = []
  for i in range(popsize): 
    vec = [random.randint(domain[0][0], domain[i][1]) for i in range(len(domain))]
    pop.append(vec)

  # 엘리트해의 개수 설정
  topelite = int(elite*popsize)

  # 메인루프
  for i in range(maxiter):
    scores = [(costf(v), v) for v in pop]
    scores.sort()
    ranked = [v for (s, v) in scores]

    # 엘리트해의 추출
    pop = ranked[0:topelite]

    # 변이와 교차의 반복. popsize만큼.
    while len(pop)<popsize:
      if random.random()<mutprob:
        #변이
        c = random.randint(0, topelite)
        pop.append(mutate(ranked[c]))

      else:
        #교차(싱글)
        c1 = random.randint(0, topelite)
        c2 = random.randint(0, topelite)
        pop.append(crossover(ranked[c1], ranked[c2]))

    # 현재까지의 최적해 출력
    print(i, scores[0][0], '--->', scores[0][1], '==>', printsolution(scores[0][1]))

  return scores[0][1]

In [17]:
geneticoptimize(domain) # cost가 점점 작아지도록 학습

0 [[7]] ---> [14, 3, 1, 14, 4, 11, 5, 5, 0, 3, 6, 2, 0, 1, 3, 2, 1, 0] ==> ['E', 'B', 'A', 'F', 'C', 'F', 'C', 'D', 'A', 'C', 'E', 'B', 'A', 'D', 'F', 'E', 'D', 'B']
1 [[7]] ---> [14, 3, 1, 14, 4, 11, 5, 5, 0, 3, 6, 2, 0, 1, 3, 2, 1, 0] ==> ['E', 'B', 'A', 'F', 'C', 'F', 'C', 'D', 'A', 'C', 'E', 'B', 'A', 'D', 'F', 'E', 'D', 'B']
2 [[2]] ---> [0, 6, 8, 10, 6, 6, 10, 7, 5, 1, 1, 0, 5, 1, 0, 1, 1, 0] ==> ['A', 'C', 'D', 'E', 'C', 'D', 'F', 'E', 'C', 'A', 'B', 'A', 'F', 'B', 'B', 'E', 'F', 'D']
3 [[2]] ---> [0, 6, 8, 10, 6, 6, 10, 7, 5, 1, 1, 0, 5, 1, 0, 1, 1, 0] ==> ['A', 'C', 'D', 'E', 'C', 'D', 'F', 'E', 'C', 'A', 'B', 'A', 'F', 'B', 'B', 'E', 'F', 'D']
4 [[0]] ---> [7, 5, 1, 3, 4, 12, 11, 9, 6, 3, 2, 0, 1, 3, 3, 2, 0, 0] ==> ['C', 'B', 'A', 'B', 'C', 'F', 'F', 'E', 'D', 'C', 'B', 'A', 'D', 'E', 'F', 'E', 'A', 'D']
5 [[0]] ---> [7, 5, 1, 3, 4, 12, 11, 9, 6, 3, 2, 0, 1, 3, 3, 2, 0, 0] ==> ['C', 'B', 'A', 'B', 'C', 'F', 'F', 'E', 'D', 'C', 'B', 'A', 'D', 'E', 'F', 'E', 'A', 'D']
6 [[0]] 

[0, 3, 2, 13, 6, 6, 5, 6, 2, 2, 2, 0, 4, 1, 1, 1, 0, 0]